# Improved Stress Level Prediction Pipeline

## Overview

This notebook implements a comprehensive data preprocessing and feature engineering pipeline for stress level prediction using multimodal wearable sensor data (Empatica E4).

**Key Improvements:**
1. **Signal Preprocessing**: Bandpass filtering, motion artifact removal, quality assessment
2. **Subject-Specific Normalization**: Z-score using rest phase baseline (handles 10x EDA variation)
3. **EDA Decomposition**: Tonic/phasic separation + SCR features (THE stress biomarker)
4. **Nonlinear HRV**: Sample Entropy, Approximate Entropy, DFA (complexity measures)
5. **Cross-Modal Synchrony**: EDA-HR correlation and coherence (multi-system coordination)
6. **Demographic Features**: Gender, age, height, weight, BMI, physical activity

**Expected Performance:**
- Current: 91% accuracy, 48% macro F1
- Target: 94-96% accuracy, 75-86% macro F1

**Runtime:** ~15-20 minutes for full pipeline

## Phase 0: Setup & Configuration

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import os
import warnings
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from tqdm.auto import tqdm

# Signal processing
from scipy import signal
from scipy.signal import butter, filtfilt, find_peaks, coherence
from scipy.interpolate import interp1d
from scipy.stats import skew, kurtosis

# Machine learning
from sklearn.model_selection import GroupKFold
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report
)
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
warnings.filterwarnings('ignore')
np.random.seed(42)
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [2]:
# Configuration
BASE_DIR = Path("/home/moh/home/Data_mining/Stress-Level-Prediction")
DATASETS_DIR = BASE_DIR / "Datasets"
OUTPUT_DIR = BASE_DIR

# File paths
SUBJECT_INFO_PATH = BASE_DIR / "subject-info.csv"
STRESS_LABELS_V1 = BASE_DIR / "Stress_Level_v1.csv"
STRESS_LABELS_V2 = BASE_DIR / "Stress_Level_v2.csv"

# Window parameters
WINDOW_SIZE = 60  # seconds
STEP_SIZE = 30    # seconds (50% overlap)
TARGET_FS = 4.0   # Hz (resampling frequency)

# Signal quality thresholds
MIN_SIGNAL_QUALITY = 0.3  # Minimum quality score to include window
MAX_MOTION_RATIO = 0.5    # Maximum motion artifact ratio
MIN_IBI_COUNT = 5         # Minimum IBI beats per window

# Data constraints (from data_constraints.txt)
DATA_CONSTRAINTS = {
    'S02': {'duplicates': {'ACC': 49545, 'BVP': 99091, 'EDA_TEMP': 6195}},
    'f07': {'missing_sensors': ['BVP', 'TEMP', 'HR', 'IBI']},
    'f14': {'split_files': ['f14_a', 'f14_b']},
    'S01': {'missing_ibi': ['ANAEROBIC']},
    'S12': {'skip': ['AEROBIC']}
}

print(f"✓ Configuration loaded")
print(f"  Base directory: {BASE_DIR}")
print(f"  Window: {WINDOW_SIZE}s, Step: {STEP_SIZE}s")
print(f"  Target sampling rate: {TARGET_FS} Hz")

✓ Configuration loaded
  Base directory: /home/moh/home/Data_mining/Stress-Level-Prediction
  Window: 60s, Step: 30s
  Target sampling rate: 4.0 Hz


## Phase 1: Signal Preprocessing Functions

In [3]:
def bandpass_filter_signal(data: np.ndarray, lowcut: float, highcut: float, 
                          fs: float, order: int = 3) -> np.ndarray:
    """Apply Butterworth bandpass filter to signal."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    
    if low <= 0:
        low = 0.001
    if high >= 1:
        high = 0.999
    
    b, a = butter(order, [low, high], btype='band')
    filtered = filtfilt(b, a, data)
    return filtered


def lowpass_filter_signal(data: np.ndarray, cutoff: float, fs: float, 
                         order: int = 3) -> np.ndarray:
    """Apply Butterworth lowpass filter to signal."""
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    if normal_cutoff >= 1:
        normal_cutoff = 0.999
    
    b, a = butter(order, normal_cutoff, btype='low')
    filtered = filtfilt(b, a, data)
    return filtered


def highpass_filter_signal(data: np.ndarray, cutoff: float, fs: float, 
                          order: int = 3) -> np.ndarray:
    """Apply Butterworth highpass filter to signal."""
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    if normal_cutoff <= 0:
        normal_cutoff = 0.001
    
    b, a = butter(order, normal_cutoff, btype='high')
    filtered = filtfilt(b, a, data)
    return filtered


def preprocess_signals(eda: np.ndarray, temp: np.ndarray, 
                      acc: np.ndarray, bvp: Optional[np.ndarray], 
                      fs: float = 4.0) -> Tuple[np.ndarray, ...]:
    """Apply bandpass filtering to all signals."""
    # EDA: Bandpass 0.01-5 Hz
    eda_clean = bandpass_filter_signal(eda, 0.01, 5.0, fs)
    
    # TEMP: Lowpass 0.5 Hz
    temp_clean = lowpass_filter_signal(temp, 0.5, fs)
    
    # ACC: Lowpass 15 Hz
    acc_clean = lowpass_filter_signal(acc, 15.0, fs)
    
    # BVP: Bandpass 0.5-8 Hz (if available)
    if bvp is not None and len(bvp) > 0:
        bvp_clean = bandpass_filter_signal(bvp, 0.5, 8.0, fs)
    else:
        bvp_clean = bvp
    
    return eda_clean, temp_clean, acc_clean, bvp_clean


def detect_motion_artifacts(acc_mag: np.ndarray, eda: np.ndarray, 
                           threshold: float = 2.0) -> Tuple[np.ndarray, float]:
    """Detect and remove motion artifacts from EDA."""
    acc_mean = np.mean(acc_mag)
    acc_std = np.std(acc_mag)
    
    # Detect high motion periods
    motion_mask = acc_mag > (acc_mean + threshold * acc_std)
    
    # Mark EDA as invalid during high motion
    eda_clean = eda.copy()
    eda_clean[motion_mask] = np.nan
    
    # Interpolate over motion artifacts
    valid_idx = ~np.isnan(eda_clean)
    if valid_idx.sum() > 2:
        eda_clean = np.interp(
            np.arange(len(eda)),
            np.where(valid_idx)[0],
            eda_clean[valid_idx]
        )
    else:
        eda_clean = eda
    
    motion_ratio = motion_mask.sum() / len(motion_mask)
    return eda_clean, motion_ratio


def assess_signal_quality(data: np.ndarray, signal_name: str = 'EDA') -> float:
    """Assess signal quality (0-1 score)."""
    quality_score = 1.0
    
    if np.std(data) < 0.01:
        quality_score *= 0.3
    
    nan_ratio = np.isnan(data).sum() / len(data)
    if nan_ratio > 0.3:
        quality_score *= 0.5
    
    if signal_name == 'EDA':
        if np.any(data < 0) or np.any(data > 60):
            quality_score *= 0.4
    elif signal_name == 'TEMP':
        if np.any(data < 20) or np.any(data > 45):
            quality_score *= 0.4
    elif signal_name == 'HR':
        valid_data = data[~np.isnan(data)]
        if len(valid_data) > 0:
            if np.any(valid_data < 30) or np.any(valid_data > 200):
                quality_score *= 0.4
    
    return quality_score


print("✓ Signal preprocessing functions defined")

✓ Signal preprocessing functions defined


## Phase 2: Subject-Specific Normalization

In [4]:
def normalize_by_subject_baseline(df: pd.DataFrame, feature_cols: List[str], 
                                 subject_col: str = 'subject', 
                                 phase_col: str = 'phase') -> pd.DataFrame:
    """
    Z-score normalize features using each subject's rest phase as baseline.
    
    This accounts for individual differences:
    - Baseline EDA: 0.5-5 µS (10x variation)
    - Baseline HR: 50-90 bpm (40 bpm variation)
    """
    normalized_df = df.copy()
    
    for subject in df[subject_col].unique():
        subject_mask = df[subject_col] == subject
        
        # Use rest phase as individual baseline
        rest_mask = subject_mask & (df[phase_col] == 'rest')
        
        if rest_mask.sum() > 0:
            baseline_mean = df.loc[rest_mask, feature_cols].mean()
            baseline_std = df.loc[rest_mask, feature_cols].std().replace(0, 1)
        else:
            baseline_mean = df.loc[subject_mask, feature_cols].mean()
            baseline_std = df.loc[subject_mask, feature_cols].std().replace(0, 1)
        
        # Z-score normalization
        normalized_df.loc[subject_mask, feature_cols] = (
            (df.loc[subject_mask, feature_cols] - baseline_mean) / baseline_std
        )
    
    return normalized_df


print("✓ Subject-specific normalization function defined")

✓ Subject-specific normalization function defined


## Phase 3: EDA Decomposition & SCR Features

In [5]:
def decompose_eda(eda_signal: np.ndarray, fs: float = 4.0) -> Tuple[np.ndarray, np.ndarray]:
    """Decompose EDA into tonic (SCL) and phasic (SCR) components."""
    # Tonic component: Low-pass < 0.05 Hz
    tonic = lowpass_filter_signal(eda_signal, 0.05, fs, order=3)
    
    # Phasic component: High-pass > 0.05 Hz
    phasic = highpass_filter_signal(eda_signal, 0.05, fs, order=3)
    
    return tonic, phasic


def extract_scr_features(phasic: np.ndarray, fs: float = 4.0) -> Dict[str, float]:
    """Extract Skin Conductance Response (SCR) features."""
    scr_features = {}
    
    # Detect SCR peaks
    peaks, properties = find_peaks(
        phasic,
        height=0.01,
        distance=int(fs * 1.0),
        prominence=0.01
    )
    
    duration_min = len(phasic) / (fs * 60)
    
    scr_features['scr_count'] = len(peaks)
    scr_features['scr_rate'] = len(peaks) / duration_min if duration_min > 0 else 0.0
    
    if len(peaks) > 0:
        amplitudes = properties['peak_heights']
        scr_features['scr_amp_mean'] = float(np.mean(amplitudes))
        scr_features['scr_amp_max'] = float(np.max(amplitudes))
        scr_features['scr_amp_std'] = float(np.std(amplitudes))
        scr_features['scr_amp_sum'] = float(np.sum(amplitudes))
    else:
        scr_features['scr_amp_mean'] = 0.0
        scr_features['scr_amp_max'] = 0.0
        scr_features['scr_amp_std'] = 0.0
        scr_features['scr_amp_sum'] = 0.0
    
    if 'widths' in properties and len(properties['widths']) > 0:
        rise_times = properties['widths'] / fs
        scr_features['scr_rise_time_mean'] = float(np.mean(rise_times))
    else:
        scr_features['scr_rise_time_mean'] = 0.0
    
    return scr_features


print("✓ EDA decomposition and SCR extraction functions defined")

✓ EDA decomposition and SCR extraction functions defined


## Phase 4: Nonlinear HRV Features

In [6]:
def validate_ibi(ibi: np.ndarray, min_count: int = 5, 
                min_rr: float = 0.3, max_rr: float = 2.0) -> Optional[np.ndarray]:
    """Validate and clean IBI data."""
    if ibi is None or len(ibi) == 0:
        return None
    
    valid_mask = (ibi >= min_rr) & (ibi <= max_rr) & ~np.isnan(ibi)
    cleaned_ibi = ibi[valid_mask]
    
    if len(cleaned_ibi) < min_count:
        return None
    
    return cleaned_ibi


def sample_entropy(data: np.ndarray, m: int = 2, r: float = 0.2) -> float:
    """Calculate Sample Entropy (SampEn)."""
    N = len(data)
    if N < m + 10:
        return np.nan
    
    r = r * np.std(data)
    
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])
    
    def _phi(m):
        patterns = [[data[j] for j in range(i, i + m)] for i in range(N - m + 1)]
        C = []
        for i in range(len(patterns)):
            count = sum(1 for j in range(len(patterns))
                       if i != j and _maxdist(patterns[i], patterns[j]) <= r)
            C.append(count)
        phi = sum(C) / (N - m + 1) / (N - m) if (N - m) > 0 else 0
        return phi
    
    phi_m = _phi(m)
    phi_m1 = _phi(m + 1)
    
    if phi_m > 0 and phi_m1 > 0:
        return -np.log(phi_m1 / phi_m)
    return np.nan


def approximate_entropy(data: np.ndarray, m: int = 2, r: float = 0.2) -> float:
    """Calculate Approximate Entropy (ApEn)."""
    N = len(data)
    if N < m + 10:
        return np.nan
    
    r = r * np.std(data)
    
    def _phi(m):
        patterns = [[data[j] for j in range(i, i + m)] for i in range(N - m + 1)]
        C = []
        for i in range(len(patterns)):
            count = sum(1 for j in range(len(patterns))
                       if np.max(np.abs(np.array(patterns[i]) - np.array(patterns[j]))) <= r)
            C.append(count / (N - m + 1))
        if all(c > 0 for c in C):
            return sum(np.log(C)) / (N - m + 1)
        return np.nan
    
    phi_m = _phi(m)
    phi_m1 = _phi(m + 1)
    
    if not np.isnan(phi_m) and not np.isnan(phi_m1):
        return abs(phi_m - phi_m1)
    return np.nan


def detrended_fluctuation_analysis(data: np.ndarray) -> Tuple[float, float]:
    """Calculate DFA alpha coefficients."""
    N = len(data)
    if N < 16:
        return np.nan, np.nan
    
    y = np.cumsum(data - np.mean(data))
    scales = np.unique(np.logspace(0.5, np.log10(N//4), 20).astype(int))
    
    F = []
    for scale in scales:
        n_segments = N // scale
        F_scale = 0
        
        for i in range(n_segments):
            segment = y[i*scale:(i+1)*scale]
            t = np.arange(len(segment))
            coeffs = np.polyfit(t, segment, 1)
            trend = np.polyval(coeffs, t)
            F_scale += np.mean((segment - trend) ** 2)
        
        if n_segments > 0:
            F.append(np.sqrt(F_scale / n_segments))
        else:
            F.append(np.nan)
    
    log_scales = np.log(scales)
    log_F = np.log(F)
    
    valid_mask = ~np.isnan(log_F)
    if valid_mask.sum() < 2:
        return np.nan, np.nan
    
    log_scales = log_scales[valid_mask]
    log_F = log_F[valid_mask]
    scales = scales[valid_mask]
    
    mask1 = (scales >= 4) & (scales <= 11)
    if np.sum(mask1) > 1:
        alpha1 = np.polyfit(log_scales[mask1], log_F[mask1], 1)[0]
    else:
        alpha1 = np.nan
    
    mask2 = scales > 11
    if np.sum(mask2) > 1:
        alpha2 = np.polyfit(log_scales[mask2], log_F[mask2], 1)[0]
    else:
        alpha2 = np.nan
    
    return alpha1, alpha2


print("✓ Nonlinear HRV functions defined")

✓ Nonlinear HRV functions defined


## Phase 5: Cross-Modal Synchrony Features

In [7]:
def cross_modal_features(eda: np.ndarray, hr: np.ndarray, 
                        temp: np.ndarray, fs: float = 4.0) -> Dict[str, float]:
    """Extract cross-modal synchrony features."""
    features = {}
    
    min_len = min(len(eda), len(hr), len(temp))
    eda = eda[:min_len]
    hr = hr[:min_len]
    temp = temp[:min_len]
    
    if min_len < 10:
        return {
            'eda_hr_xcorr_max': np.nan,
            'eda_temp_xcorr_max': np.nan,
            'hr_temp_xcorr_max': np.nan,
            'eda_hr_coherence_lf': np.nan,
            'eda_hr_coherence_hf': np.nan,
        }
    
    # Normalize signals
    eda_norm = (eda - np.mean(eda)) / (np.std(eda) + 1e-6)
    hr_norm = (hr - np.mean(hr)) / (np.std(hr) + 1e-6)
    temp_norm = (temp - np.mean(temp)) / (np.std(temp) + 1e-6)
    
    # Cross-correlation
    xcorr_eda_hr = np.correlate(eda_norm, hr_norm, mode='same')
    xcorr_eda_temp = np.correlate(eda_norm, temp_norm, mode='same')
    xcorr_hr_temp = np.correlate(hr_norm, temp_norm, mode='same')
    
    features['eda_hr_xcorr_max'] = float(np.max(np.abs(xcorr_eda_hr)))
    features['eda_temp_xcorr_max'] = float(np.max(np.abs(xcorr_eda_temp)))
    features['hr_temp_xcorr_max'] = float(np.max(np.abs(xcorr_hr_temp)))
    
    # Coherence
    if min_len >= 64:
        f, Cxy_eda_hr = coherence(eda, hr, fs=fs, nperseg=min(64, min_len))
        
        lf_mask = (f >= 0.04) & (f <= 0.15)
        features['eda_hr_coherence_lf'] = (
            float(np.mean(Cxy_eda_hr[lf_mask])) if lf_mask.sum() > 0 else np.nan
        )
        
        hf_mask = (f >= 0.15) & (f <= 0.4)
        features['eda_hr_coherence_hf'] = (
            float(np.mean(Cxy_eda_hr[hf_mask])) if hf_mask.sum() > 0 else np.nan
        )
    else:
        features['eda_hr_coherence_lf'] = np.nan
        features['eda_hr_coherence_hf'] = np.nan
    
    return features


print("✓ Cross-modal synchrony functions defined")

✓ Cross-modal synchrony functions defined


## Phase 6: Demographic Feature Integration

In [8]:
def load_demographic_data(subject_info_path: Path) -> pd.DataFrame:
    """Load and process demographic data from subject-info.csv."""
    df = pd.read_csv(subject_info_path)
    
    demographics = pd.DataFrame()
    demographics['subject'] = df['Info']
    demographics['gender'] = df['Gender'].map({'M': 1, 'F': 0})
    demographics['age'] = pd.to_numeric(df['Age'], errors='coerce')
    demographics['height'] = pd.to_numeric(df['Height (cm)'], errors='coerce')
    demographics['weight'] = pd.to_numeric(df['Weight (kg)'], errors='coerce')
    demographics['bmi'] = demographics['weight'] / ((demographics['height'] / 100) ** 2)
    demographics['physical_activity'] = df['Does physical activity regularly?'].map({'Yes': 1, 'No': 0})
    demographics['protocol_version'] = df['Protocol'].map({'V1': 1, 'V2': 0})
    
    # Fill missing values
    for col in ['age', 'height', 'weight', 'bmi']:
        demographics[col] = demographics[col].fillna(demographics[col].median())
    
    demographics['gender'] = demographics['gender'].fillna(0)
    demographics['physical_activity'] = demographics['physical_activity'].fillna(0)
    demographics['protocol_version'] = demographics['protocol_version'].fillna(0)
    
    return demographics


print("✓ Demographic data loading function defined")

✓ Demographic data loading function defined


## Phase 7: Feature Extraction Functions

In [9]:
def hampel_filter(data: np.ndarray, window_size: int = 5, n_sigmas: float = 3) -> np.ndarray:
    """Apply Hampel filter to remove outliers."""
    filtered = data.copy()
    k = 1.4826
    
    for i in range(len(data)):
        start = max(0, i - window_size)
        end = min(len(data), i + window_size + 1)
        window = data[start:end]
        
        median = np.median(window)
        mad = k * np.median(np.abs(window - median))
        
        if np.abs(data[i] - median) > n_sigmas * mad:
            filtered[i] = median
    
    return filtered


def linear_slope(data: np.ndarray, fs: float) -> float:
    """Calculate linear slope of signal."""
    if len(data) < 2:
        return 0.0
    t = np.arange(len(data)) / fs
    slope = np.polyfit(t, data, 1)[0]
    return float(slope)


def extract_eda_features(eda: np.ndarray, fs: float = 4.0) -> Dict[str, float]:
    """Extract comprehensive EDA features."""
    features = {}
    
    clean = hampel_filter(eda)
    tonic, phasic = decompose_eda(clean, fs)
    scr_feats = extract_scr_features(phasic, fs)
    
    # Tonic features
    features['eda_scl_mean'] = float(np.mean(tonic))
    features['eda_scl_std'] = float(np.std(tonic))
    features['eda_scl_slope'] = linear_slope(tonic, fs)
    features['eda_scl_range'] = float(np.max(tonic) - np.min(tonic))
    
    # Phasic features
    features['eda_phasic_mean'] = float(np.mean(phasic))
    features['eda_phasic_std'] = float(np.std(phasic))
    features['eda_phasic_energy'] = float(np.sum(phasic ** 2))
    
    # SCR features
    features.update(scr_feats)
    
    # Basic statistics
    features['eda_mean'] = float(np.mean(clean))
    features['eda_std'] = float(np.std(clean))
    features['eda_min'] = float(np.min(clean))
    features['eda_max'] = float(np.max(clean))
    features['eda_range'] = float(np.max(clean) - np.min(clean))
    features['eda_skew'] = float(skew(clean))
    features['eda_kurt'] = float(kurtosis(clean))
    features['eda_slope'] = linear_slope(clean, fs)
    
    return features


def extract_hrv_features(ibi: Optional[np.ndarray], window_duration: float = 60.0) -> Dict[str, float]:
    """Extract comprehensive HRV features."""
    features = {}
    
    ibi_clean = validate_ibi(ibi, min_count=MIN_IBI_COUNT)
    
    if ibi_clean is None or len(ibi_clean) < MIN_IBI_COUNT:
        feature_names = [
            'hrv_mean_rr', 'hrv_std_rr', 'hrv_rmssd', 'hrv_sdnn',
            'hrv_pnn50', 'hrv_mean_hr', 'hrv_std_hr',
            'hrv_lf', 'hrv_hf', 'hrv_lf_hf_ratio',
            'hrv_sampen', 'hrv_apen', 'hrv_dfa_alpha1', 'hrv_dfa_alpha2'
        ]
        return {name: np.nan for name in feature_names}
    
    rr = ibi_clean * 1000
    
    # Time-domain
    features['hrv_mean_rr'] = float(np.mean(rr))
    features['hrv_std_rr'] = float(np.std(rr))
    
    diff_rr = np.diff(rr)
    features['hrv_rmssd'] = float(np.sqrt(np.mean(diff_rr ** 2)))
    features['hrv_sdnn'] = float(np.std(rr))
    
    nn50 = np.sum(np.abs(diff_rr) > 50)
    features['hrv_pnn50'] = float(nn50 / len(diff_rr) * 100) if len(diff_rr) > 0 else 0.0
    
    hr = 60000 / rr
    features['hrv_mean_hr'] = float(np.mean(hr))
    features['hrv_std_hr'] = float(np.std(hr))
    
    # Frequency-domain
    if len(rr) >= 10:
        t_rr = np.cumsum(ibi_clean)
        t_uniform = np.arange(0, t_rr[-1], 0.25)
        rr_interp = np.interp(t_uniform, t_rr, rr)
        
        freqs, psd = signal.welch(rr_interp, fs=4.0, nperseg=min(256, len(rr_interp)))
        
        lf_mask = (freqs >= 0.04) & (freqs <= 0.15)
        features['hrv_lf'] = float(np.trapz(psd[lf_mask], freqs[lf_mask])) if lf_mask.sum() > 0 else 0.0
        
        hf_mask = (freqs >= 0.15) & (freqs <= 0.4)
        features['hrv_hf'] = float(np.trapz(psd[hf_mask], freqs[hf_mask])) if hf_mask.sum() > 0 else 0.0
        
        features['hrv_lf_hf_ratio'] = (
            float(features['hrv_lf'] / features['hrv_hf']) if features['hrv_hf'] > 0 else 0.0
        )
    else:
        features['hrv_lf'] = np.nan
        features['hrv_hf'] = np.nan
        features['hrv_lf_hf_ratio'] = np.nan
    
    # Nonlinear
    if len(rr) >= 10:
        features['hrv_sampen'] = sample_entropy(rr, m=2, r=0.2)
        features['hrv_apen'] = approximate_entropy(rr, m=2, r=0.2)
        features['hrv_dfa_alpha1'], features['hrv_dfa_alpha2'] = detrended_fluctuation_analysis(rr)
    else:
        features['hrv_sampen'] = np.nan
        features['hrv_apen'] = np.nan
        features['hrv_dfa_alpha1'] = np.nan
        features['hrv_dfa_alpha2'] = np.nan
    
    return features


def extract_temp_features(temp: np.ndarray, fs: float = 4.0) -> Dict[str, float]:
    """Extract temperature features."""
    return {
        'temp_mean': float(np.mean(temp)),
        'temp_std': float(np.std(temp)),
        'temp_min': float(np.min(temp)),
        'temp_max': float(np.max(temp)),
        'temp_range': float(np.max(temp) - np.min(temp)),
        'temp_slope': linear_slope(temp, fs)
    }


def extract_acc_features(acc: np.ndarray, fs: float = 4.0) -> Dict[str, float]:
    """Extract accelerometer features."""
    acc_mag = np.linalg.norm(acc, axis=1) if acc.ndim > 1 else np.abs(acc)
    
    return {
        'acc_mean': float(np.mean(acc_mag)),
        'acc_std': float(np.std(acc_mag)),
        'acc_min': float(np.min(acc_mag)),
        'acc_max': float(np.max(acc_mag)),
        'acc_range': float(np.max(acc_mag) - np.min(acc_mag)),
        'acc_energy': float(np.sum(acc_mag ** 2))
    }


def extract_hr_features(hr: np.ndarray) -> Dict[str, float]:
    """Extract heart rate features."""
    valid_hr = hr[~np.isnan(hr)]
    
    if len(valid_hr) > 0:
        return {
            'hr_mean': float(np.mean(valid_hr)),
            'hr_std': float(np.std(valid_hr)),
            'hr_min': float(np.min(valid_hr)),
            'hr_max': float(np.max(valid_hr)),
            'hr_range': float(np.max(valid_hr) - np.min(valid_hr))
        }
    else:
        return {k: np.nan for k in ['hr_mean', 'hr_std', 'hr_min', 'hr_max', 'hr_range']}


print("✓ Feature extraction functions defined")

✓ Feature extraction functions defined


## Phase 8: Data Loading Functions

In [10]:
def load_sensor_file(file_path: Path) -> Tuple[np.ndarray, float]:
    """Load a sensor file from Empatica E4 format."""
    if not file_path.exists():
        return None, None
    
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    if len(lines) < 3:
        return None, None
    
    fs = float(lines[1].strip())
    data = np.array([float(line.strip()) for line in lines[2:]])
    
    return data, fs


def load_ibi_file(file_path: Path) -> Optional[np.ndarray]:
    """Load IBI file (inter-beat intervals)."""
    if not file_path.exists():
        return None
    
    try:
        df = pd.read_csv(file_path, names=['timestamp', 'ibi'])
        ibi = df['ibi'].values
        return ibi if len(ibi) > 0 else None
    except:
        return None


def resample_signal(data: np.ndarray, original_fs: float, target_fs: float) -> np.ndarray:
    """Resample signal to target frequency."""
    if original_fs == target_fs:
        return data
    
    duration = len(data) / original_fs
    n_samples = int(duration * target_fs)
    
    t_original = np.arange(len(data)) / original_fs
    t_target = np.arange(n_samples) / target_fs
    
    if data.ndim == 1:
        resampled = np.interp(t_target, t_original, data)
    else:
        resampled = np.zeros((n_samples, data.shape[1]))
        for i in range(data.shape[1]):
            resampled[:, i] = np.interp(t_target, t_original, data[:, i])
    
    return resampled


def load_stress_labels() -> Dict[str, pd.DataFrame]:
    """Load stress level labels for all subjects."""
    labels = {}
    
    if STRESS_LABELS_V1.exists():
        df_v1 = pd.read_csv(STRESS_LABELS_V1)
        for _, row in df_v1.iterrows():
            subject = row.iloc[0]
            labels[subject] = row.iloc[1:].to_dict()
    
    if STRESS_LABELS_V2.exists():
        df_v2 = pd.read_csv(STRESS_LABELS_V2)
        for _, row in df_v2.iterrows():
            subject = row.iloc[0]
            labels[subject] = row.iloc[1:].to_dict()
    
    return labels


def map_stress_score_to_class(score: float) -> str:
    """Map self-reported stress score [0-10] to stress class."""
    if pd.isna(score):
        return 'unknown'
    
    if score <= 2:
        return 'no_stress'
    elif score <= 5:
        return 'low_stress'
    elif score <= 7:
        return 'moderate_stress'
    else:
        return 'high_stress'


print("✓ Data loading functions defined")

✓ Data loading functions defined


## Phase 9: Main Processing Pipeline

This cell processes all subjects and extracts features. Due to length, this has been simplified. Run this to build the dataset.

In [11]:
def process_subject_protocol(subject: str, protocol: str, phase: str,
                            stress_labels: Dict) -> List[Dict]:
    """Process one subject-protocol-phase combination."""
    # Skip special cases
    if subject == 'S12' and protocol == 'AEROBIC':
        return []
    
    subject_dir = DATASETS_DIR / protocol / subject / phase
    if not subject_dir.exists():
        return []
    
    # Load sensor files
    eda_raw, eda_fs = load_sensor_file(subject_dir / 'EDA.csv')
    temp_raw, temp_fs = load_sensor_file(subject_dir / 'TEMP.csv')
    hr_raw, hr_fs = load_sensor_file(subject_dir / 'HR.csv')
    bvp_raw, bvp_fs = load_sensor_file(subject_dir / 'BVP.csv')
    ibi_raw = load_ibi_file(subject_dir / 'IBI.csv')
    
    # Load ACC
    acc_file = subject_dir / 'ACC.csv'
    if acc_file.exists():
        with open(acc_file, 'r') as f:
            lines = f.readlines()
        acc_fs = float(lines[1].strip())
        acc_data = []
        for line in lines[2:]:
            values = line.strip().split(',')
            if len(values) == 3:
                acc_data.append([float(v) for v in values])
        acc_raw = np.array(acc_data) / 64.0  # Convert to g
    else:
        acc_raw, acc_fs = None, None
    
    if eda_raw is None or temp_raw is None:
        return []
    
    # Handle f07 special case
    if subject == 'f07':
        bvp_raw = None
        hr_raw = None
        ibi_raw = None
    
    # Resample signals
    eda = resample_signal(eda_raw, eda_fs, TARGET_FS)
    temp = resample_signal(temp_raw, temp_fs, TARGET_FS)
    
    if hr_raw is not None:
        hr = resample_signal(hr_raw, hr_fs, TARGET_FS)
    else:
        hr = np.full(len(eda), np.nan)
    
    if acc_raw is not None:
        acc = resample_signal(acc_raw, acc_fs, TARGET_FS)
    else:
        acc = np.zeros((len(eda), 3))
    
    if bvp_raw is not None:
        bvp = resample_signal(bvp_raw, bvp_fs, TARGET_FS)
    else:
        bvp = None
    
    # Apply preprocessing
    eda_clean, temp_clean, acc_clean, bvp_clean = preprocess_signals(
        eda, temp, acc, bvp, fs=TARGET_FS
    )
    
    # Motion artifact removal
    acc_mag = np.linalg.norm(acc_clean, axis=1)
    eda_clean, motion_ratio = detect_motion_artifacts(acc_mag, eda_clean)
    
    # Extract windows
    window_samples = int(WINDOW_SIZE * TARGET_FS)
    step_samples = int(STEP_SIZE * TARGET_FS)
    
    windows = []
    n_samples = min(len(eda_clean), len(temp_clean), len(hr), len(acc_clean))
    
    for start in range(0, n_samples - window_samples + 1, step_samples):
        end = start + window_samples
        
        eda_win = eda_clean[start:end]
        temp_win = temp_clean[start:end]
        hr_win = hr[start:end]
        acc_win = acc_clean[start:end]
        
        # Quality assessment
        eda_quality = assess_signal_quality(eda_win, 'EDA')
        temp_quality = assess_signal_quality(temp_win, 'TEMP')
        hr_quality = assess_signal_quality(hr_win, 'HR')
        
        if eda_quality < MIN_SIGNAL_QUALITY or temp_quality < MIN_SIGNAL_QUALITY:
            continue
        
        # Extract features
        features = {}
        features.update(extract_eda_features(eda_win, TARGET_FS))
        features.update(extract_hrv_features(ibi_raw, WINDOW_SIZE))
        features.update(extract_temp_features(temp_win, TARGET_FS))
        features.update(extract_acc_features(acc_win, TARGET_FS))
        features.update(extract_hr_features(hr_win))
        features.update(cross_modal_features(eda_win, hr_win, temp_win, TARGET_FS))
        
        # Metadata
        features['subject'] = subject
        features['protocol'] = protocol
        features['phase'] = phase
        features['window_id'] = len(windows)
        features['signal_quality_eda'] = eda_quality
        features['signal_quality_temp'] = temp_quality
        features['signal_quality_hr'] = hr_quality
        features['motion_ratio'] = motion_ratio
        
        # Assign label
        if protocol == 'STRESS' and subject in stress_labels:
            stress_score = stress_labels[subject].get(phase, np.nan)
            features['stress_score'] = stress_score
            features['stress_class'] = map_stress_score_to_class(stress_score)
        else:
            features['stress_score'] = np.nan
            if protocol in ['AEROBIC', 'ANAEROBIC']:
                features['stress_class'] = 'no_stress' if phase == 'rest' else protocol.lower()
            else:
                features['stress_class'] = 'unknown'
        
        windows.append(features)
    
    return windows


print("✓ Subject processing function defined")

✓ Subject processing function defined


In [12]:
def build_dataset() -> pd.DataFrame:
    """Build complete dataset from all subjects and protocols."""
    print("\n" + "="*80)
    print("BUILDING DATASET")
    print("="*80)
    
    # Load stress labels
    print("\n[1/3] Loading stress labels...")
    stress_labels = load_stress_labels()
    print(f"  Loaded labels for {len(stress_labels)} subjects")
    
    # Get all subjects
    subjects = []
    for protocol_dir in DATASETS_DIR.iterdir():
        if protocol_dir.is_dir():
            for subject_dir in protocol_dir.iterdir():
                if subject_dir.is_dir():
                    subjects.append((subject_dir.name, protocol_dir.name))
    
    subjects = list(set(subjects))
    subjects.sort()
    
    print(f"\n[2/3] Processing {len(subjects)} subject-protocol combinations...")
    all_windows = []
    
    for subject, protocol in tqdm(subjects, desc="Processing"):
        subject_dir = DATASETS_DIR / protocol / subject
        
        for phase_dir in subject_dir.iterdir():
            if phase_dir.is_dir():
                phase = phase_dir.name
                windows = process_subject_protocol(subject, protocol, phase, stress_labels)
                all_windows.extend(windows)
    
    print(f"  Extracted {len(all_windows)} windows")
    
    # Convert to DataFrame
    print("\n[3/3] Building DataFrame...")
    df = pd.DataFrame(all_windows)
    
    print(f"\n  Dataset shape: {df.shape}")
    print(f"  Subjects: {df['subject'].nunique()}")
    if 'stress_class' in df.columns:
        print(f"\n  Stress class distribution:")
        print(df['stress_class'].value_counts())
    
    print("\n" + "="*80)
    return df


print("✓ Dataset building function defined")

✓ Dataset building function defined


In [13]:
# Build the dataset (this will take 15-20 minutes)
print("Starting dataset construction...\n")
dataset = build_dataset()
print("\nDataset construction complete!")

Starting dataset construction...


BUILDING DATASET

[1/3] Loading stress labels...
  Loaded labels for 36 subjects

[2/3] Processing 100 subject-protocol combinations...


Processing:   0%|          | 0/100 [00:00<?, ?it/s]

  Extracted 0 windows

[3/3] Building DataFrame...

  Dataset shape: (0, 0)


KeyError: 'subject'

## Phase 10: Add Demographics & Normalization

In [ ]:
# Add demographics
print("\nAdding demographic features...")
demographics = load_demographic_data(SUBJECT_INFO_PATH)
dataset = dataset.merge(demographics, on='subject', how='left')
print(f"Dataset shape: {dataset.shape}")

# Apply normalization
print("\nApplying subject-specific normalization...")
exclude_cols = ['subject', 'protocol', 'phase', 'window_id', 'stress_score', 'stress_class',
                'signal_quality_eda', 'signal_quality_temp', 'signal_quality_hr', 'motion_ratio']
feature_cols = [col for col in dataset.columns if col not in exclude_cols]

dataset = normalize_by_subject_baseline(dataset, feature_cols, 'subject', 'phase')
print("✓ Normalization complete")

# Save dataset
output_file = OUTPUT_DIR / "improved_stress_dataset.csv"
dataset.to_csv(output_file, index=False)
print(f"\n✓ Dataset saved to: {output_file}")

## Phase 11: Model Training

In [ ]:
# Prepare data
valid_classes = ['no_stress', 'low_stress', 'moderate_stress', 'high_stress']
dataset_filtered = dataset[dataset['stress_class'].isin(valid_classes)].copy()

print(f"Filtered dataset: {dataset_filtered.shape}")
print(f"\nClass distribution:")
print(dataset_filtered['stress_class'].value_counts())

exclude_cols = ['subject', 'protocol', 'phase', 'window_id', 'stress_score', 'stress_class',
                'signal_quality_eda', 'signal_quality_temp', 'signal_quality_hr', 'motion_ratio']
feature_cols = [col for col in dataset_filtered.columns if col not in exclude_cols]

X = dataset_filtered[feature_cols].fillna(0).values
y = dataset_filtered['stress_class'].values
groups = dataset_filtered['subject'].values

le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"\nFeatures: {X.shape}")
print(f"Classes: {le.classes_}")

In [ ]:
# Train with cross-validation
print("\n" + "="*80)
print("5-FOLD CROSS-VALIDATION")
print("="*80)

xgb_params = {
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 200,
    'objective': 'multi:softmax',
    'num_class': len(le.classes_),
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'n_jobs': -1
}

gkf = GroupKFold(n_splits=5)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y_encoded, groups), 1):
    print(f"\n--- Fold {fold} ---")
    
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]
    
    model = xgb.XGBClassifier(**xgb_params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    y_pred = model.predict(X_val)
    
    acc = accuracy_score(y_val, y_pred)
    f1_macro = f1_score(y_val, y_pred, average='macro')
    f1_weighted = f1_score(y_val, y_pred, average='weighted')
    
    print(f"Accuracy:    {acc:.4f}")
    print(f"Macro F1:    {f1_macro:.4f}")
    print(f"Weighted F1: {f1_weighted:.4f}")
    
    fold_results.append({
        'fold': fold,
        'accuracy': acc,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'model': model
    })

# Summary
print("\n" + "="*80)
print("SUMMARY")
print("="*80)

avg_acc = np.mean([r['accuracy'] for r in fold_results])
avg_f1_macro = np.mean([r['f1_macro'] for r in fold_results])
avg_f1_weighted = np.mean([r['f1_weighted'] for r in fold_results])

print(f"\nAverage Accuracy:    {avg_acc:.4f}")
print(f"Average Macro F1:    {avg_f1_macro:.4f}")
print(f"Average Weighted F1: {avg_f1_weighted:.4f}")

print("\n### BASELINE vs IMPROVED:")
print(f"Accuracy:  90.8% → {avg_acc*100:.1f}% ({(avg_acc-0.908)*100:+.1f}pp)")
print(f"Macro F1:  47.6% → {avg_f1_macro*100:.1f}% ({(avg_f1_macro-0.476)*100:+.1f}pp)")
print("\n" + "="*80)

## Phase 12: Train Final Model & Feature Importance

In [ ]:
# Train final model
print("\nTraining final model on full dataset...")
final_model = xgb.XGBClassifier(**xgb_params)
final_model.fit(X, y_encoded)

model_path = OUTPUT_DIR / "xgboost_stress_model.json"
final_model.save_model(str(model_path))
print(f"✓ Model saved to: {model_path}")

# Feature importance
importances = final_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': importances
}).sort_values('importance', ascending=False)

importance_path = OUTPUT_DIR / "feature_importance.csv"
feature_importance_df.to_csv(importance_path, index=False)
print(f"✓ Feature importance saved to: {importance_path}")

print(f"\nTop 20 features:")
print(feature_importance_df.head(20))

# Plot
plt.figure(figsize=(12, 8))
top_20 = feature_importance_df.head(20)
plt.barh(range(len(top_20)), top_20['importance'])
plt.yticks(range(len(top_20)), top_20['feature'])
plt.xlabel('Importance')
plt.title('Top 20 Most Important Features')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "feature_importance.png", dpi=300)
plt.show()

print("\n" + "="*80)
print("PIPELINE COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. improved_stress_dataset.csv")
print("  2. xgboost_stress_model.json")
print("  3. feature_importance.csv")
print("  4. feature_importance.png")
print("\n" + "="*80)